In [1]:
import json
import os
import pandas as pd
import numpy as np
import urllib.request
import requests
import string
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas_datareader.data as web
import datetime

In [83]:
key = os.environ.get('FMP_KEY')
ticker = 'GOOG'
#Income statement
income = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=120&apikey={key}')
incomes = income.json()

#Cashflow
cf = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=120&apikey={key}')
cfs = cf.json()

#Balance Sheet
bs = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?limit=120&apikey={key}')
bss = bs.json()

#EV
evv = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}?limit=120&apikey={key}')
enterprise = evv.json()


In [84]:
revenue_growth = (incomes[0]['revenue']-incomes[1]['revenue'])/incomes[1]['revenue']
print(revenue_growth)

0.1830008989979462


In [85]:
#Income statement
net_income = incomes[0]['netIncome']
income_statement = pd.DataFrame.from_dict(incomes[0], orient='index')
income_statement = income_statement[5:26]
income_statement.columns =['current_year']
income_statement['as_%_of_revenue']= income_statement / income_statement.iloc[0]

#Forecast for the next 5 years
income_statement['next_year'] = (income_statement['current_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']
income_statement['next_1_year'] = (income_statement['next_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']
income_statement['next_2_year'] = (income_statement['next_1_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']
income_statement['next_3_year'] = (income_statement['next_2_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']
income_statement['next_4_year'] = (income_statement['next_3_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']
income_statement['next_5_year'] = (income_statement['next_4_year']['revenue']*(1+revenue_growth)) * income_statement['as_%_of_revenue']

In [86]:
#Balance Sheeet
balance_sheet = pd.DataFrame.from_dict(bss[0], orient='index')
balance_sheet = balance_sheet[5:-2]
balance_sheet.columns =['current_year']
balance_sheet['as_%_of_revenue']= balance_sheet / income_statement['current_year'].iloc[0]

#Forecast for the next 5 years
balance_sheet['next_year'] = (income_statement['current_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']
balance_sheet['next_1_year'] = (income_statement['next_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']
balance_sheet['next_2_year'] = (income_statement['next_1_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']
balance_sheet['next_3_year'] = (income_statement['next_2_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']
balance_sheet['next_4_year'] = (income_statement['next_3_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']
balance_sheet['next_5_year'] = (income_statement['next_4_year']['revenue']*(1+revenue_growth)) * balance_sheet['as_%_of_revenue']

In [87]:
#Forcast next years cash flow
CF_forecast = {}
CF_forecast['next_year'] = {}
CF_forecast['next_year']['netIncome'] = income_statement['next_year']['netIncome']
CF_forecast['next_year']['inc_depreciation'] = income_statement['next_year']['depreciationAndAmortization'] - income_statement['current_year']['depreciationAndAmortization']
CF_forecast['next_year']['inc_receivables'] = balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']
CF_forecast['next_year']['inc_inventory'] = balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']
CF_forecast['next_year']['inc_payables'] = balance_sheet['next_year']['accountPayables'] - balance_sheet['current_year']['accountPayables']
CF_forecast['next_year']['CF_operations'] = CF_forecast['next_year']['netIncome'] + CF_forecast['next_year']['inc_depreciation'] + (CF_forecast['next_year']['inc_receivables'] * -1) + (CF_forecast['next_year']['inc_inventory'] *-1) + CF_forecast['next_year']['inc_payables']
CF_forecast['next_year']['CAPEX'] = balance_sheet['next_year']['propertyPlantEquipmentNet'] - balance_sheet['current_year']['propertyPlantEquipmentNet'] + income_statement['next_year']['depreciationAndAmortization']

CF_forecast['next_year']['FCF'] = CF_forecast['next_year']['CAPEX'] + CF_forecast['next_year']['CF_operations']

In [90]:
#Forecast year 2 in cash flow
CF_forecast['next_2_year'] = {}
CF_forecast['next_2_year']['netIncome'] = income_statement['next_2_year']['netIncome']

CF_forecast['next_2_year']['inc_depreciation'] = income_statement['next_2_year']['depreciationAndAmortization'] - income_statement['next_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['inc_receivables'] = balance_sheet['next_2_year']['netReceivables'] - balance_sheet['next_year']['netReceivables']
CF_forecast['next_2_year']['inc_inventory'] = balance_sheet['next_2_year']['inventory'] - balance_sheet['next_year']['inventory']
CF_forecast['next_2_year']['inc_payables'] = balance_sheet['next_2_year']['accountPayables'] - balance_sheet['next_year']['accountPayables']
CF_forecast['next_2_year']['CF_operations'] = CF_forecast['next_2_year']['netIncome'] + CF_forecast['next_2_year']['inc_depreciation'] + (CF_forecast['next_2_year']['inc_receivables'] * -1) + (CF_forecast['next_2_year']['inc_inventory'] *-1) + CF_forecast['next_2_year']['inc_payables']
CF_forecast['next_2_year']['CAPEX'] = balance_sheet['next_2_year']['propertyPlantEquipmentNet'] - balance_sheet['next_year']['propertyPlantEquipmentNet'] + income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['FCF'] = CF_forecast['next_2_year']['CAPEX'] + CF_forecast['next_2_year']['CF_operations']

In [91]:
#Forecast year 3 in cash flow
CF_forecast['next_3_year'] = {}
CF_forecast['next_3_year']['netIncome'] = income_statement['next_3_year']['netIncome']

CF_forecast['next_3_year']['inc_depreciation'] = income_statement['next_3_year']['depreciationAndAmortization'] - income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['inc_receivables'] = balance_sheet['next_3_year']['netReceivables'] - balance_sheet['next_2_year']['netReceivables']
CF_forecast['next_3_year']['inc_inventory'] = balance_sheet['next_3_year']['inventory'] - balance_sheet['next_2_year']['inventory']
CF_forecast['next_3_year']['inc_payables'] = balance_sheet['next_3_year']['accountPayables'] - balance_sheet['next_2_year']['accountPayables']
CF_forecast['next_3_year']['CF_operations'] = CF_forecast['next_3_year']['netIncome'] + CF_forecast['next_3_year']['inc_depreciation'] + (CF_forecast['next_3_year']['inc_receivables'] * -1) + (CF_forecast['next_3_year']['inc_inventory'] *-1) + CF_forecast['next_3_year']['inc_payables']
CF_forecast['next_3_year']['CAPEX'] = balance_sheet['next_3_year']['propertyPlantEquipmentNet'] - balance_sheet['next_2_year']['propertyPlantEquipmentNet'] + income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['FCF'] = CF_forecast['next_3_year']['CAPEX'] + CF_forecast['next_3_year']['CF_operations']

In [92]:
#Forecast year 4 in cash flow
CF_forecast['next_4_year'] = {}
CF_forecast['next_4_year']['netIncome'] = income_statement['next_4_year']['netIncome']

CF_forecast['next_4_year']['inc_depreciation'] = income_statement['next_4_year']['depreciationAndAmortization'] - income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['inc_receivables'] = balance_sheet['next_4_year']['netReceivables'] - balance_sheet['next_3_year']['netReceivables']
CF_forecast['next_4_year']['inc_inventory'] = balance_sheet['next_4_year']['inventory'] - balance_sheet['next_3_year']['inventory']
CF_forecast['next_4_year']['inc_payables'] = balance_sheet['next_4_year']['accountPayables'] - balance_sheet['next_3_year']['accountPayables']
CF_forecast['next_4_year']['CF_operations'] = CF_forecast['next_4_year']['netIncome'] + CF_forecast['next_4_year']['inc_depreciation'] + (CF_forecast['next_4_year']['inc_receivables'] * -1) + (CF_forecast['next_4_year']['inc_inventory'] *-1) + CF_forecast['next_4_year']['inc_payables']
CF_forecast['next_4_year']['CAPEX'] = balance_sheet['next_4_year']['propertyPlantEquipmentNet'] - balance_sheet['next_3_year']['propertyPlantEquipmentNet'] + income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['FCF'] = CF_forecast['next_4_year']['CAPEX'] + CF_forecast['next_4_year']['CF_operations']


In [93]:
#Forecast year 5 in cash flow
CF_forecast['next_5_year'] = {}
CF_forecast['next_5_year']['netIncome'] = income_statement['next_5_year']['netIncome']

CF_forecast['next_5_year']['inc_depreciation'] = income_statement['next_5_year']['depreciationAndAmortization'] - income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['inc_receivables'] = balance_sheet['next_5_year']['netReceivables'] - balance_sheet['next_4_year']['netReceivables']
CF_forecast['next_5_year']['inc_inventory'] = balance_sheet['next_5_year']['inventory'] - balance_sheet['next_4_year']['inventory']
CF_forecast['next_5_year']['inc_payables'] = balance_sheet['next_5_year']['accountPayables'] - balance_sheet['next_4_year']['accountPayables']
CF_forecast['next_5_year']['CF_operations'] = CF_forecast['next_5_year']['netIncome'] + CF_forecast['next_5_year']['inc_depreciation'] + (CF_forecast['next_5_year']['inc_receivables'] * -1) + (CF_forecast['next_5_year']['inc_inventory'] *-1) + CF_forecast['next_5_year']['inc_payables']
CF_forecast['next_5_year']['CAPEX'] = balance_sheet['next_5_year']['propertyPlantEquipmentNet'] - balance_sheet['next_4_year']['propertyPlantEquipmentNet'] + income_statement['next_5_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['FCF'] = CF_forecast['next_5_year']['CAPEX'] + CF_forecast['next_5_year']['CF_operations']

In [94]:
#add the forecasted cash flows into a Pandas
CF_forec = pd.DataFrame.from_dict(CF_forecast,orient='columns')

#add below option to format the dataframe with thousand separators
pd.options.display.float_format = '{:,.0f}'.format

CF_forec

,next_year,next_2_quarter,next_3_year,next_2_year,next_4_year,next_5_year
netIncome,"40,627,799,874",nan,"67,263,355,465","56,858,245,435","79,572,609,985","94,134,469,147"
inc_depreciation,"2,155,933,591",nan,"3,569,362,061","5,567,681,308","4,222,558,527","4,995,290,534"
inc_receivables,"5,031,060,715",nan,"8,329,420,404","12,992,674,180","9,853,711,826","11,656,949,949"
inc_inventory,"182,817,898",nan,"302,673,177","472,125,764","358,062,641","423,588,426"
inc_payables,"1,017,667,999",nan,"1,684,850,388","2,628,119,493","1,993,179,524","2,357,933,168"
CF_operations,"38,587,522,851",nan,"63,885,474,333","51,589,246,292","75,576,573,568","89,407,154,475"
CAPEX,"29,416,430,635",nan,"48,701,820,832","59,480,292,581","57,614,297,827","68,157,766,125"
FCF,"68,003,953,486",nan,"112,587,295,165","111,069,538,873","133,190,871,396","157,564,920,599"


In [95]:
def interest_coveraga_and_RF(ticker):
  IS= requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={key}').json()
  EBIT= IS[0]['ebitda'] - IS[0]['depreciationAndAmortization'] 
  interest_expense = IS[0]['interestExpense']
  interest_coverage_ratio = EBIT/interest_expense

    #RF
  start = datetime.datetime(2019, 9, 26)
        
  end= datetime.datetime.today().strftime('%Y-%m-%d')
  #end = datetime.datetime(2020, 9, 30)

  Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
  RF = float(Treasury.iloc[-1])
  RF = RF/100
  return [RF,interest_coverage_ratio]

In [67]:
#Cost of debt
def cost_of_debt(ticker, RF,interest_coverage_ratio):
  if interest_coverage_ratio > 8.5:
    #Rating is AAA
    credit_spread = 0.0063
  if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
    credit_spread = 0.0078
  if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
    credit_spread = 0.0098
  if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
    credit_spread = 0.0108
  if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
    credit_spread = 0.0122
  if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
    credit_spread = 0.0156
  if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
    credit_spread = 0.02
  if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
    credit_spread = 0.0240
  if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
    credit_spread = 0.0351
  if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
    credit_spread = 0.0421
  if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
    credit_spread = 0.0515
  if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
    credit_spread = 0.0820
  if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
    credit_spread = 0.0864
  if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
    credit_spread = 0.1134
  if interest_coverage_ratio <=  0.2:
    #Rating is D
    credit_spread = 0.1512
  
  cost_of_debt = RF + credit_spread
  return cost_of_debt

In [68]:
def costofequity(ticker):


  #RF
  start = datetime.datetime(2019, 9, 26)
  end= datetime.datetime.today().strftime('%Y-%m-%d')

  Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
  RF = float(Treasury.iloc[-1])
  RF = RF/100

#Beta

  beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{ticker}?apikey={key}')
  beta = beta.json()
  beta = float(beta['profile']['beta'])


  #Market Return
  start = datetime.datetime(2019, 7, 10)
  end= datetime.datetime.today().strftime('%Y-%m-%d')

  SP500 = web.DataReader(['sp500'], 'fred', start, end)
      #Drop all Not a number values using drop method.
  SP500.dropna(inplace = True)

  SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1
    
  cost_of_equity = RF+(beta*(SP500yearlyreturn - RF))
  return cost_of_equity

In [69]:
#effective tax rate and capital structure
def wacc(ticker):
  FR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={key}').json()

  ETR = FR[0]['effectiveTaxRate']

# 
  BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=quarter&apikey={key}').json()



  Debt_to = BS[0]['totalDebt'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])
  equity_to = BS[0]['totalStockholdersEquity'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])

  WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)
  return WACC

In [70]:
RF_and_IntCov = interest_coveraga_and_RF(ticker)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(ticker)
kd = cost_of_debt(ticker,RF,interest_coverage_ratio)
wacc_company = wacc(ticker)
print('wacc of ' + ticker + ' is ' + str((wacc_company*100))+'%')

wacc of GOOG is 13.731845107955012%


In [71]:
#FCF List of CFs for each year
FCF_List = CF_forec.iloc[-1].values.tolist()
npv = np.npv(wacc_company,FCF_List)

/Users/abdirahmanali/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
#Terminal value
LTGrowth = 0.02

Terminal_value = (CF_forecast['next_5_year']['FCF'] * (1+ LTGrowth)) /(wacc_company - LTGrowth)

Terminal_value_Discounted = Terminal_value/(1+wacc_company)**4
Terminal_value_Discounted

818775891592.3005

In [73]:
target_equity_value = Terminal_value_Discounted + npv
debt = balance_sheet['current_year']['totalDebt']
target_value = target_equity_value - debt
numbre_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{ticker}?apikey={key}').json()
numbre_of_shares = numbre_of_shares[0]['numberOfShares']

target_price_per_share = round(target_value/numbre_of_shares,2)
target_price_per_share

print(ticker + ' forecasted price per stock is ' + ' $'+str(target_price_per_share) )
print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_growth) + ' Cost of Capital: ' + str(wacc_company) )
print('perpetuity growth: ' + str(LTGrowth) )

GOOG forecasted price per stock is  $1807.32
the forecast is based on the following assumptions: revenue growth: 0.1830008989979462 Cost of Capital: 0.13731845107955012
perpetuity growth: 0.02


#### Model estimates that each stock is worth 1631. This is cheap since its trading below the current price of 1781. Some of the challenges with DCF model is the amount of estimations that we need to perform. We need to estimate how much the company is going to grow. If we get that wrong, our model will not give good results.

#### When we change the revenue_growth rate, it automatically changes the overall value of each stock. That is why it is good practice to estimate 3 scenerios for the growth values. One with an optimistic growth rate, likely outcome, and a pessimistic outlook. Then you get the average of the 3 scenerios to get an estimated value.